# Maximum Independent Set
Quantum Optimization Benchmark Library: https://doi.org/10.48550/arXiv.2504.03832

Hardware: Apple M1 Pro (8 cores, 16 GB memory)

## Benchmark

In [3]:
using Quicopt, LinearAlgebra, Printf
using CPUTime
include("../../src/utils.jl")
Base.show(io::IO, f::Float64) = @printf(io, "%1.5f", f)

In [4]:
PATH = "../../instances/max_independent_set/"
filename = "R_1000_005_1.gph";

In [5]:
# load graph from file
graph = load_dimacs_graph_matrix(PATH * filename)

# system size
N = size(graph)[1]

1000

In [6]:
# number of edges
size(findall(x -> x==1, UpperTriangular(graph)), 1)

24670

In [ ]:
# Lagrange multiplier
λ = 1.0

# cost function
local_fields = -ones(N) ./ λ + 0.5 .* (ones(N)' * graph)';
couplings = -0.5 .* graph

tensor_problem = TensorProblem(local_fields, couplings);

### Fast Solution

In [10]:
# parameters
T = 2.0^7
tol = 1e-1;

In [11]:
# run once to force precompilation
mf_sol = solve(tensor_problem, T, atol=tol, rtol=tol)
sol_vec = sign.(mf_sol.u[end][3, :]);

In [12]:
# map from Ising to binary
bit_string = (1 .- sol_vec) ./ 2

# check violated clauses
num_violated_clauses = Int(0.5 * bit_string' * graph * bit_string)
num_violated_clauses |> println

# check max. indep. set
sum(bit_string) |> println

0
106.00000


In [13]:
false_idxs = findall(x -> x == 1., bit_string .* (graph * bit_string))
violated_bonds = Set([[idx, intersect(false_idxs, findall(x -> x == 1, graph[idx, :]))[1]] |> sort for idx in false_idxs])

# flip violated bonds
flip_idxs = [bond[1] for bond in violated_bonds]
bit_string[flip_idxs] .= 0.
bit_string' * graph * bit_string |> println
sum(bit_string) |> println

0.00000
106.00000


In [14]:
# get CPU time
@CPUtime begin
    mf_sol = solve(tensor_problem, T, atol=tol, rtol=tol)
    sol_vec = sign.(mf_sol.u[end][3, :])
    false_idxs = findall(x -> x == 1., bit_string .* (graph * bit_string))
    violated_bonds = Set([[idx, intersect(false_idxs, findall(x -> x == 1, graph[idx, :]))[1]] |> sort for idx in false_idxs])
    flip_idxs = [bond[1] for bond in violated_bonds]
    bit_string[flip_idxs] .= 0.
end;

elapsed CPU time: 10.04948 seconds


### Slow Solution

In [15]:
# parameters
T = 600.0
tol = 1e-4;

In [16]:
# run once to force precompilation
mf_sol = solve(tensor_problem, T, atol=tol, rtol=tol)
sol_vec = sign.(mf_sol.u[end][3, :]);

In [17]:
# map from Ising to binary
bit_string = (1 .- sol_vec) ./ 2

# check violated clauses
num_violated_clauses = Int(0.5 * bit_string' * graph * bit_string)
num_violated_clauses |> println

# check max. indep. set
sum(bit_string) |> println

4
115.00000


In [18]:
false_idxs = findall(x -> x == 1., bit_string .* (graph * bit_string))
violated_bonds = Set([[idx, intersect(false_idxs, findall(x -> x == 1, graph[idx, :]))[1]] |> sort for idx in false_idxs])

# flip violated bonds
flip_idxs = [bond[1] for bond in violated_bonds]
bit_string[flip_idxs] .= 0.
bit_string' * graph * bit_string |> println
sum(bit_string) |> println

0.00000
111.00000


In [19]:
# get CPU time
@CPUtime begin
    mf_sol = solve(tensor_problem, T, atol=tol, rtol=tol)
    sol_vec = sign.(mf_sol.u[end][3, :])
    false_idxs = findall(x -> x == 1., bit_string .* (graph * bit_string))
    violated_bonds = Set([[idx, intersect(false_idxs, findall(x -> x == 1, graph[idx, :]))[1]] |> sort for idx in false_idxs])
    flip_idxs = [bond[1] for bond in violated_bonds]
    bit_string[flip_idxs] .= 0.
end;

elapsed CPU time: 87.25617 seconds
